# Set-up API key

In [1]:
import openai

with open("../API_KEY", "r") as f:
    key = f.read()

openai.api_key = key

# General Plan:
## TODO: rewrite this and all following Markdown cells
- Make a function that converts a python function into the json representation required by openai
- Then allow for making a chain where GPT mostly creates functions for itself:
    - Get gpt to make a function
    - Get gpt to save that function in a folder along with the json representation of the function
    - Automatically include all tools in the folder (i.e. using the json representations, but then being able to call the functions)
    

In [2]:
# Putting a few long functions in here if they aren't important for the main story
import demo_functions

# Other things that will be used later
import json
import numpy as np
import inspect
import os
import importlib.util
from typing import Callable

## First, let's test out the function calling

Make a very basic function first

# TODO: Make this function add some extra info that the LLM doesn't already have to make it more obvious that the function call can be helpful in some way

In [3]:
# Simple example function for initial testing
def get_weather_report(day_of_week: int, weather_type: str, temperature: float = 10.0) -> str:
    """
    Converts information about weather into a string representation.

    Args:
        day_of_week (int): The day of the week from 0 to 6.
        weather_type (str): The type of weather, can be "sunny", "rainy", or "windy".
        temperature (float, optional): Temperature in Celsius. Defaults to 10.0.

    Returns:
        str: A string representation of the weather report.
    """
    return f'For the {day_of_week}th day of the week, the weather is predicted to be {weather_type} with a max temperature of {temperature}'


In [4]:
WEATHER_REPORT_DESCRIPTION = {
    "name": "get_weather_report",
    "description": "Converts information about weather into a string representation",
    "parameters": {
        "type": "object",
        "properties": {
            "day_of_week": {
                "type": "number",
                "description": "The day of the week from 0 to 6",
            },
            "weather_type": {"type": "string", "enum": ["sunny", "rainy", "windy"]},
            "temperature": {"type": "number", "description": "Temperature in Celsius. Defaults to 10.0."},
        },
        "required": ["day_of_week", "weather_type"],
    },
}


Test using this with the new openai `function_call` parameter

In [5]:
first_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": "It's currently a sunny 36.4C this Tuesday, can you give me a weather report?"}],
    functions=[
        WEATHER_REPORT_DESCRIPTION,
    ],
    function_call="auto",
)
first_response

<OpenAIObject chat.completion id=chatcmpl-7S7oxKY3SeQqFLbk8ehkjGDGKdSSl at 0x1e18c0a97f0> JSON: {
  "id": "chatcmpl-7S7oxKY3SeQqFLbk8ehkjGDGKdSSl",
  "object": "chat.completion",
  "created": 1686937603,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_weather_report",
          "arguments": "{\n  \"day_of_week\": 2,\n  \"weather_type\": \"sunny\",\n  \"temperature\": 36.4\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 118,
    "completion_tokens": 36,
    "total_tokens": 154
  }
}

We can see that a `function_call` was made (rather than returning `content`)

Now, we need to handle the function call

In [7]:
# Extract the returned message
first_message = first_response['choices'][0]['message']

# From that message, get the name of the function called
function_name = first_message["function_call"]["name"]

# Also get the arguments (this is a string representation of a JSON dict of arguments)
arguments = first_message['function_call'].get('arguments')

# Make the call to the function with the arguments that the LLM decided
function_response = get_weather_report(
    **json.loads(arguments),  # Unpack the arguments as keyword: value pairs
)

# Look at what the function returned
print(f'Function output:\n{function_response}')

Function output:
For the 2th day of the week, the weather is predicted to be sunny with a max temperature of 36.4


And then, pass that result back to the LLM

In [8]:
function_response_message = {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }

second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "It's currently 36.4C this Tuesday, can you give me a weather report?"},
        first_message,  # Pass in the context (i.e. that it previously requested a function_call
        function_response_message,  # And the result of that function call
    ],
)
second_response

<OpenAIObject chat.completion id=chatcmpl-7S7pB9SYLdEvKVloxXuTTBnguGPqG at 0x1e18c95fb30> JSON: {
  "id": "chatcmpl-7S7pB9SYLdEvKVloxXuTTBnguGPqG",
  "object": "chat.completion",
  "created": 1686937617,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Celsius. It's going to be a sunny day with a temperature of 36.4 degrees Celsius. Enjoy the weather!"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 97,
    "completion_tokens": 26,
    "total_tokens": 123
  }
}

In [9]:
second_response['choices'][0]['message']['content']

"Celsius. It's going to be a sunny day with a temperature of 36.4 degrees Celsius. Enjoy the weather!"

TODO: Add some description of what we see overall here

# Automate making the JSON description of existing functions you want GPT to use

## Can try make conversion using fixed code 
(For this, you really have to think about all the possible formats the existing functions may have)

In [10]:
get_json_representation = demo_functions.get_json_representation_using_code
get_json_representation(get_weather_report)

{'name': 'get_weather_report',
 'description': 'Converts information about weather into a string representation.',
 'parameters': {'type': 'object',
  'properties': {'day_of_week': {'type': 'number',
    'description': 'The day of the week from 0 to 6.'},
   'weather_type': {'type': 'string',
    'description': 'The type of weather, can be "sunny", "rainy", or "windy".'},
   'temperature': {'type': 'number',
    'description': 'Temperature in Celsius. Defaults to 10.0.'}},
  'required': ['day_of_week', 'weather_type']}}

## Or can make GPT do the conversion for us
Won't be 100% deterministic, but will handle varying existing code

In [11]:
import inspect
from typing import Callable

SYSTEM_PROMPT_JSON_REP = '''Your job is to convert a python function into a json representation with a specific form.
For example, given this function:
```
def get_weather_report(day_of_week: int, weather_type: str, temperature: float = 10.0) -> str:
    """
    Converts information about weather into a string representation.

    Args:
        day_of_week (int): The day of the week from 0 to 6.
        weather_type (str): The type of weather, can be "sunny", "rainy", or "windy".
        temperature (float, optional): Temperature in Celsius. Defaults to 10.0.

    Returns:
        str: A string representation of the weather report.
    """
    return f'For the {day_of_week}th day of the week, the weather is predicted to be {weather_type} with a max temperature of {temperature}'
```

You should return:
```
{
    "name": "get_weather_report",
    "description": "Converts information about weather into a string representation",
    "parameters": {
        "type": "object",
        "properties": {
            "day_of_week": {
                "type": "number",
                "description": "The day of the week from 0 to 6",
            },
            "weather_type": {"type": "string", "enum": ["sunny", "rainy", "windy"]},
            "temperature": {"type": "number", "description": "Temperature in Celsius. Defaults to 10.0."},
        },
        "required": ["day_of_week", "weather_type"],
    },
}
```
Return the JSON ONLY.''' 


def get_json_representation(func: Callable) -> dict:
    """
    Uses the openai.ChatCompletion.create endpoint to return the necessary imports for a function supplied as a string.

    Args:
        function_str (str): The function for which to get the necessary imports, supplied as a string.

    Returns:
        str: A string of the necessary imports for the function.
    """
    # Set up the request to OpenAI
    parameters = {
        'model': 'gpt-3.5-turbo-0613',
        'temperature': 0.0,  # Deterministic output (most probable next word every time)
        'messages': [
            {"role": "system", "content": SYSTEM_PROMPT_JSON_REP},  # Pass in the initial instructions
            {"role": "user", "content": inspect.getsource(func)}  # Then give the func code
        ]
    }

    # Ask OpenAI to make the conversion to JSON format
    response = openai.ChatCompletion.create(**parameters)

    # Extract the assistant's response
    assistant_response = response['choices'][0]['message']['content']

    return json.loads(assistant_response)

In [12]:
# Note: inspect just lets you see the source code of a function
print(inspect.getsource(get_weather_report))

def get_weather_report(day_of_week: int, weather_type: str, temperature: float = 10.0) -> str:
    """
    Converts information about weather into a string representation.

    Args:
        day_of_week (int): The day of the week from 0 to 6.
        weather_type (str): The type of weather, can be "sunny", "rainy", or "windy".
        temperature (float, optional): Temperature in Celsius. Defaults to 10.0.

    Returns:
        str: A string representation of the weather report.
    """
    return f'For the {day_of_week}th day of the week, the weather is predicted to be {weather_type} with a max temperature of {temperature}'



In [13]:
get_json_representation(get_weather_report)

{'name': 'get_weather_report',
 'description': 'Converts information about weather into a string representation',
 'parameters': {'type': 'object',
  'properties': {'day_of_week': {'type': 'number',
    'description': 'The day of the week from 0 to 6'},
   'weather_type': {'type': 'string', 'enum': ['sunny', 'rainy', 'windy']},
   'temperature': {'type': 'number',
    'description': 'Temperature in Celsius. Defaults to 10.0.'}},
  'required': ['day_of_week', 'weather_type']}}

Not surprising that it does well on this since we gave it this as an example as well, but we'll use it again later and see that it continues to work

# TODO: Try out forcing gpt to fill a fake function_call with info (maybe better at returning json content with smaller prompt?)

# Let's structure how we can store these tools

- Store each function/tool in it's own `.py` file with the same name as the function
    - We'll need to make sure we include the necessary `imports`
- Store the JSON descriptions of the functions in a `.json` file with the same name as the function
- Load the JSON descriptions from the `.json` files
- Import and run the necessary functions from the `.py` files

### Let's think about how we can save this example function

In [14]:
def test_func(d: dict, a: int, b: float) -> str:
    """
    Calculates the sine of the product of a and b and adds it to the dictionary d with the key 'new_val'.

    Args:
      d: A dictionary to which the new value will be added.
      a: An integer value.
      b: A float value.

    Returns:
      A JSON string representation of the updated dictionary d.
    """
    d['new_val'] = np.sin(a*b)
    return json.dumps(d)

test_func({}, 1, 3)

'{"new_val": 0.1411200080598672}'

If we want to save this to it's own file, we are going to need to include the imports for `json` and `np`.

It should look something like 

%test_func.py
```
import json
import numpy as np

def test_func(d: dict, a: int, b: float) -> str:
    """
    Calculates the sine of the product of a and b and adds it to the dictionary d with the key 'new_val'.

    Args:
      d: A dictionary to which the new value will be added.
      a: An integer value.
      b: A float value.

    Returns:
      A JSON string representation of the updated dictionary d.
    """
    d['new_val'] = np.sin(a*b)
    return json.dumps(d)

test_func({}, 1, 3)
```

How can we automate the process of figuring out what imports are necessary (not very simple since some imports require knowledge that e.g. `np` is just the shorthand name of `numpy`)

### Figure out the necessary imports for a function

This is again a fairly difficult task to accomplish with code, but we can get GPT to help us. 

In [15]:
SYSTEM_PROMPT_IMPORTS_REQUIRED = '''Your job is to return ONLY the python imports that would be required to run the function given by the user. For example:
-----
The user has given you this function:
```
def some_function(a, b, **kwargs):
    c = np.sin(a+b)
    result = {'input_a': a, 'input_b': b, 'res': c}
    return json.dumps(result)
```
You should return:
```
import numpy as np
import json
```
-----

Do not include ANY other text other than the imports required. If no imports are required return "No imports required".
'''

def get_imports_for_function(function_str: str) -> str:
    """
    Uses the openai.ChatCompletion.create endpoint to return the necessary imports for a function supplied as a string.

    Args:
        function_str (str): The function for which to get the necessary imports, supplied as a string.

    Returns:
        str: A string of the necessary imports for the function.
    """
    parameters = {
        'model': 'gpt-3.5-turbo-0613',
        'temperature': 0.0,
        'messages': [
            {"role": "system", "content": SYSTEM_PROMPT_IMPORTS_REQUIRED},
            {"role": "user", "content": function_str}
        ]
    }

    response = openai.ChatCompletion.create(**parameters)

    # Extract the assistant's response
    assistant_response = response['choices'][0]['message']['content']

    return assistant_response

In [16]:
func_str = inspect.getsource(test_func)
print(func_str)

def test_func(d: dict, a: int, b: float) -> str:
    """
    Calculates the sine of the product of a and b and adds it to the dictionary d with the key 'new_val'.

    Args:
      d: A dictionary to which the new value will be added.
      a: An integer value.
      b: A float value.

    Returns:
      A JSON string representation of the updated dictionary d.
    """
    d['new_val'] = np.sin(a*b)
    return json.dumps(d)



In [17]:
imports = get_imports_for_function(func_str)
print(imports)

import numpy as np
import json


GPT has figured out the imports for us :)

Let's check what happens if there are no imports required

In [18]:
def test_func2(a: float, b: float) -> float:
    """
    Multiplies two float values and returns the result.

    Args:
      a: A float value.
      b: A float value.

    Returns:
      The product of a and b.
    """
    return a*b

In [19]:
get_imports_for_function(inspect.getsource(test_func2))

'No imports required.'

Perfect, that's what we asked GPT to do in this case

### Now let's save to file

In [20]:
def save_func_and_json(func: Callable, func_folder: str) -> None:
    """
    Saves the given function and its JSON representation to a subfolder of func_folder.

    Args:
        func (Callable): The function to save.
        func_folder (str): The path to the folder where the function and JSON representation will be saved.
    """
    func_str = inspect.getsource(func)
    # Get the imports required for the function
    imports_string = get_imports_for_function(func_str)
    if 'no imports required' in imports_string.lower():
        imports_string = ''
    else:
        imports_string += '\n\n\n'
        
    file_contents = imports_string+func_str
    
    # Save function to .py file
    with open(f"{func_folder}/{func.__name__}.py", "w") as f:
        f.write(file_contents)

    # Save JSON representation to .json file
    with open(f"{func_folder}/{func.__name__}.json", "w") as f:
        json.dump(get_json_representation(func), f)

In [21]:
import os
FUNCTIONS_FOLDER = 'functions'
os.makedirs(FUNCTIONS_FOLDER, exist_ok=True)

save_func_and_json(test_func, func_folder=FUNCTIONS_FOLDER)
save_func_and_json(test_func2, func_folder=FUNCTIONS_FOLDER)

And now we have those functions saved in a nice format

### Loading the functions for use by GPT

In [22]:
def load_json_descriptions(directory: str) -> dict:
    """
    Loads the contents of all the .json files in a directory.

    Args:
        directory (str): The directory to load .json files from.

    Returns:
        dict: A dictionary where the key is the name of the json file (excluding the .json extension) and the value is the content of the .json file.
    """
    json_data = {}

    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        # Check if the file is a .json file
        if filename.endswith(".json"):
            # Remove the .json extension from the filename
            name = filename[:-5]
            # Open the .json file and load its contents
            with open(os.path.join(directory, filename), 'r') as f:
                data = json.load(f)
            # Add the data to the dictionary
            json_data[name] = data

    return json_data


In [23]:
function_descriptions = load_json_descriptions(FUNCTIONS_FOLDER)
print(f"Saved functions are: {function_descriptions.keys()}\n\n")
# Using json.dumps just to make the output look nicer for us mere humans
test_func_description = json.dumps(function_descriptions['test_func'], indent=4)
print(f"test_func description: {test_func_description}")

Saved functions are: dict_keys(['multiply_list', 'multiply_numbers', 'test_func', 'test_func2'])


test_func description: {
    "name": "test_func",
    "description": "Calculates the sine of the product of a and b and adds it to the dictionary d with the key 'new_val'.",
    "parameters": {
        "type": "object",
        "properties": {
            "d": {
                "type": "object",
                "description": "A dictionary to which the new value will be added."
            },
            "a": {
                "type": "integer",
                "description": "An integer value."
            },
            "b": {
                "type": "number",
                "description": "A float value."
            }
        },
        "required": [
            "d",
            "a",
            "b"
        ]
    },
    "return": {
        "type": "string",
        "description": "A JSON string representation of the updated dictionary d."
    }
}


### And now to use a function from a file

In [24]:
def load_function_from_file(folder: str, filename: str):
    """
    Loads a function from a .py file.

    Args:
        directory (str): The directory where the .py file is located.
        filename (str): The name of the .py file (excluding the .py extension).

    Returns:
        function: The function contained in the .py file.
    """
    # Create the path to the .py file
    file_path = os.path.join(folder, filename + ".py")

    # Load the spec of the module
    spec = importlib.util.spec_from_file_location(filename, file_path)

    # Create a module from the spec
    module = importlib.util.module_from_spec(spec)

    # Execute the module to get the function
    spec.loader.exec_module(module)

    # Get the function from the module
    function = getattr(module, filename)

    return function

In [25]:
function_descriptions = load_json_descriptions(FUNCTIONS_FOLDER)
funcs = {}
for func_name in function_descriptions:
    funcs[func_name] = load_function_from_file(FUNCTIONS_FOLDER, func_name)
funcs

{'multiply_list': <function multiply_list.multiply_list(numbers: List[float]) -> float>,
 'multiply_numbers': <function multiply_numbers.multiply_numbers(numbers: List[float]) -> float>,
 'test_func': <function test_func.test_func(d: dict, a: int, b: float) -> str>,
 'test_func2': <function test_func2.test_func2(a: float, b: float) -> float>}

In [26]:
# Test with a similar example to what we did before
funcs['test_func']({}, 1, 4)

'{"new_val": -0.7568024953079282}'

# Now let's provide a way for GPT to make it's own function, save it, and use it in its next response!

- Make a write python code function (the function should just take a description of what the code needs to do so that a separate prompt can be used to actually generate the code)

# TODO: Maybe can improve the code generator by making it fill in a fake function that takes args for 'signature', 'docstring', 'code body' or something like that?

In [27]:
SYSTEM_PROMPT_MAKE_NEW_FUNCTION = '''You are an expert python coder that will be tasked with generating the code to go in a .py file for a single function given some specific information from the user.
You will be provided:
    - function_name: The name to give the new function
    - arg_descriptions: Descriptions of all the arguments the function should take (if their types are missing, try to infer them)
    - description: What the function should do with the given arguments

When generating the new function you should follow these rules:
    - Include ONLY the text that will be in the python file (e.g. starting with `import ...` unless no imports are necessary in which case, starting with `def ...`)
    - Do NOT include any plain text explanation at the end of the written code
    - Use the latest python programming techniques and best practices
    - Use the latest/best python libraries when appropriate (e.g. if plotting, use `plotly` instead of `matplotlib` because plotly is better library even though matplotlib is better known)
    - Always include a google style docstring
    - Include type hints for the inputs and output
    - Include all necessary imports
'''


def make_new_function(function_name: str, arg_descriptions: str, description: str):
    """
    Use this if an existing function doesn't exist, and it would be helpful to have a new function to complete a task. 
    The new function will be made to carry out the task described in the `description` given the arguments described by `arg_descriptions`
    
    Args:
        function_name: Name to give the new function (should follow python naming conventions)
        arg_descriptions: A description of any arguments that the function should take (including type, and default value if appropriate)
        description: A description of what the function should do (including the what it should output)
    """
    parameters = {
        'model': 'gpt-3.5-turbo-0613',
        'temperature': 0.0,
        'messages': [
            {"role": "system", "content": SYSTEM_PROMPT_MAKE_NEW_FUNCTION},
            {"role": "user", "content": f"function_name: {function_name}\narg_descriptions: {arg_descriptions}\ndescription: {description}"}
        ]
    }

    response = openai.ChatCompletion.create(**parameters)

    # Extract the assistant's response
    function_code = response['choices'][0]['message']['content']
    return function_code
    

## We need the JSON description again

In [28]:
make_new_function_description = get_json_representation(make_new_function)
make_new_function_description

{'name': 'make_new_function',
 'description': "Use this if an existing function doesn't exist, and it would be helpful to have a new function to complete a task. The new function will be made to carry out the task described in the `description` given the arguments described by `arg_descriptions`",
 'parameters': {'type': 'object',
  'properties': {'function_name': {'type': 'string'},
   'arg_descriptions': {'type': 'string'},
   'description': {'type': 'string'}},
  'required': ['function_name', 'arg_descriptions', 'description']}}

## Let's test that out

We'll ask for something that GPT is bad at doing by itself (like basic math)

In [43]:
# user_request = "I'd like you to multiply a list of numbers together and return the total value, the list is [3,2,6,3,6,5,4,3,6]."
user_request = "What is the result of multiplying together all of these numbers [3,2,6,3,6,5,4,3,6]?"

SYSTEM_PROMPT_GENERAL = '''You are a helpful AI assistant. 
When responding you should follow these rules:
 - You should ONLY consider using the functions provided (e.g. do not just assume you can use `python`)
 - You should always use functions as an intermediate step to respond to the user when appropriate (e.g. when being asked to do math, use a function to do the calculation)
 - If there is a missing function that would be useful, make a call to `make_new_function` to create it BEFORE responding to the user
'''

response_requesting_new_func = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT_GENERAL},
        {"role": "user", "content": user_request},
    ],
    functions=[
        make_new_function_description,
    ],
    function_call="auto",  
    # function_call={"name": "make_new_function"},  # Force using the function
)
response_requesting_new_func

<OpenAIObject chat.completion id=chatcmpl-7S7wZMoV0sMoTiB1E8BKU07e1s5SU at 0x1e18cd077d0> JSON: {
  "id": "chatcmpl-7S7wZMoV0sMoTiB1E8BKU07e1s5SU",
  "object": "chat.completion",
  "created": 1686938075,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "make_new_function",
          "arguments": "{\n  \"function_name\": \"multiply_numbers\",\n  \"arg_descriptions\": \"a list of numbers\",\n  \"description\": \"Multiply together all the numbers in the given list\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 238,
    "completion_tokens": 45,
    "total_tokens": 283
  }
}

GPT-3.5 has a bit of a problem of hallucinating functions that don't exist (e.g. It often assumes it has a `python` function if you let it decide what to do automatically)
If needs me, we can force it to use the `make_new_function` call by specifying `function_call={"name": "make_new_function"}`

### Now let's make the requested function and save to file
# TODO: Refactor the write to file earlier so that I don't need duplicates here

In [44]:
def sanitize_python_code(code: str) -> str:
    """
    Sanitizes a string containing Python code. If the code is surrounded by markdown triple backticks, they are removed.
    Also, language identifiers immediately following the opening backticks (like 'python' or 'py') are removed.

    Args:
        code (str): The string containing Python code.

    Returns:
        str: The sanitized Python code.
    """
    # Check if the string starts and ends with triple backticks
    if code.startswith("```") and code.endswith("```"):
        # Remove the triple backticks from the start and end of the string
        code = code[3:-3]
        
    # Further check if the string starts with "python" or "py", which is common in markdown code blocks
    if code.lstrip().startswith(("python", "py")):
        # Find the first newline character and remove everything before it
        code = code[code.find('\n')+1:]

    return code

def write_to_py_file(folder: str, file_name: str, file_contents: str):
    # Save function to .py file
    file_contents = sanitize_python_code(file_contents)
    with open(f"{folder}/{file_name}.py", "w") as f:
        f.write(file_contents)

def write_description_to_json_file(folder: str, func: Callable):
    # Save JSON representation to .json file
    with open(f"{folder}/{func.__name__}.json", "w") as f:
        json.dump(get_json_representation(func), f)
        
def write_generated_func_to_file(folder: str, func_name: str, generated_file_contents: str):
    write_to_py_file(folder, func_name, generated_file_contents)
    func = load_function_from_file(folder, func_name)
    write_description_to_json_file(folder, func)

In [45]:
message = response_requesting_new_func['choices'][0]['message']
message

<OpenAIObject at 0x1e18d3d12b0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "make_new_function",
    "arguments": "{\n  \"function_name\": \"multiply_numbers\",\n  \"arg_descriptions\": \"a list of numbers\",\n  \"description\": \"Multiply together all the numbers in the given list\"\n}"
  }
}

In [46]:
new_func_arguments_description = message['function_call'].get('arguments')
print(f'new function request: {new_func_arguments_description}')

new function request: {
  "function_name": "multiply_numbers",
  "arg_descriptions": "a list of numbers",
  "description": "Multiply together all the numbers in the given list"
}


In [47]:
new_file_contents = make_new_function(
    **json.loads(new_func_arguments_description),
)
print(f'make_new_function output:\n{new_file_contents}')

make_new_function output:
```python
from typing import List

def multiply_numbers(numbers: List[float]) -> float:
    """
    Multiply together all the numbers in the given list.

    Args:
        numbers: A list of numbers.

    Returns:
        The product of all the numbers in the list.
    """
    product = 1
    for num in numbers:
        product *= num
    return product
```


In [48]:
new_func_name = json.loads(message['function_call']['arguments']).get('function_name')
write_generated_func_to_file(FUNCTIONS_FOLDER, new_func_name, new_file_contents)
print(f'Funcion written to file')

Funcion written to file


In [49]:
func_descriptions = load_json_descriptions(FUNCTIONS_FOLDER)
func_descriptions

{'multiply_list': {'name': 'multiply_list',
  'description': 'Multiply a list of numbers together and return the total value',
  'parameters': {'type': 'object',
   'properties': {'numbers': {'type': 'array', 'items': {'type': 'number'}}},
   'required': ['numbers']},
  'returns': {'type': 'number',
   'description': 'The product of all the numbers in the list'}},
 'multiply_numbers': {'name': 'multiply_numbers',
  'description': 'Multiply together all the numbers in the given list',
  'parameters': {'type': 'object',
   'properties': {'numbers': {'type': 'array', 'items': {'type': 'number'}}},
   'required': ['numbers']},
  'return': {'type': 'number',
   'description': 'The product of all the numbers in the list'}},
 'test_func': {'name': 'test_func',
  'description': "Calculates the sine of the product of a and b and adds it to the dictionary d with the key 'new_val'.",
  'parameters': {'type': 'object',
   'properties': {'d': {'type': 'object',
     'description': 'A dictionary to 

In [50]:
response_using_new_func = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613", 
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT_GENERAL},
        {"role": "user", "content": user_request},
        # message,
        # {
        #     "role": "function",
        #     "name": called_function,
        #     "content": function_response,
        # },
    ],
    functions=[
        make_new_function_description,
        *func_descriptions.values(),
    ],
    function_call='auto',
)
print(f"Next response from Chat model:\n{response_using_new_func['choices'][0]['message']}")

Next response from Chat model:
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "multiply_list",
    "arguments": "{\n  \"numbers\": [3, 2, 6, 3, 6, 5, 4, 3, 6]\n}"
  }
}


**NOTE:** We don't want to follow the standard format here of telling GPT what the function returned, we just want to pretend that we are again answering the original question, only now we have the new function available

In [51]:
# Extract the returned message
message_using_func = response_using_new_func['choices'][0]['message']

# From that message, get the name of the function called
function_name = message_using_func["function_call"]["name"]

# Also get the arguments (this is a string representation of a JSON dict of arguments)
arguments = message_using_func['function_call'].get('arguments')

# Make the call to the function with the arguments that the LLM decided
requested_func = load_function_from_file(FUNCTIONS_FOLDER, function_name)
function_response = requested_func(
    **json.loads(arguments),  # Unpack the arguments as keyword: value pairs
)

# Look at what the function returned
print(f'Function output:\n{function_response}')

Function output:
233280.0


And then, pass that result back to the LLM

In [52]:
function_response_message = {
            "role": "function",
            "name": function_name,
            "content": str(function_response),
        }

final_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT_GENERAL},
        {"role": "user", "content": user_request},
        message_using_func,  # We'll keep this context, but don't need to include when it made the new func
        function_response_message,  # And the result of that function call
    ],
)
final_response

<OpenAIObject chat.completion id=chatcmpl-7S7yKGolOs1u6kTazgiqcBvpStmo1 at 0x1e18ccffd10> JSON: {
  "id": "chatcmpl-7S7yKGolOs1u6kTazgiqcBvpStmo1",
  "object": "chat.completion",
  "created": 1686938184,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The result of multiplying together all of the numbers [3, 2, 6, 3, 6, 5, 4, 3, 6] is 233280."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 197,
    "completion_tokens": 41,
    "total_tokens": 238
  }
}

In [53]:
final_response['choices'][0]['message']['content']

'The result of multiplying together all of the numbers [3, 2, 6, 3, 6, 5, 4, 3, 6] is 233280.'